<a href="https://colab.research.google.com/github/gmsharpe/edumore/blob/master/terraform_aws_google_colab_getting_started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Terraform, AWS Athena & Google Colab

## Download and Install Terraform

In [1]:
%%bash
# instructions from https://learn.hashicorp.com/tutorials/terraform/install-cli?in=terraform/aws-get-started
sudo apt-get update && sudo apt-get install -y gnupg software-properties-common curl
curl -fsSL https://apt.releases.hashicorp.com/gpg | sudo apt-key add -
sudo apt-add-repository "deb [arch=amd64] https://apt.releases.hashicorp.com $(lsb_release -cs) main"
sudo apt-get update && sudo apt-get install terraform

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:13 http://ppa

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


## Download and Install the AWS CLI

In [2]:
%%bash

# download the installation file
curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"

unzip -q awscliv2.zip

sudo ./aws/install


You can now run: /usr/local/bin/aws --version


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 43.4M  100 43.4M    0     0  23.1M      0  0:00:01  0:00:01 --:--:-- 23.0M



## Verify the Installation of the AWS CLI

In [3]:
%%bash
aws -version


usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help

aws: error: the following arguments are required: command



## Configure the AWS Client with your Credentials

Using `aws configure`, you next need to set your credentials and defaults for the AWS CLI.

For more detailed instructions, see [Quick configuration with aws configure](https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-quickstart.html).

In [4]:
# importing os module
import os
  
path = "/content/terraform_tutorial"
  
if not os.path.exists(path):
  os.mkdir(path)

os.chdir(path)
main_tf = open("main.tf","w")
variables_tf = open("variables.tf","w")

variables = """
locals {
  database_s3_bucket_name = "yellow-taxi-ny-983794"
  query_results_s3_bucket_name = "yellow-taxi-ny-983794-workgroup-query-results"
}
"""

main_body = """
terraform {
  required_providers {
    aws = {
      source  = "hashicorp/aws"
      version = "~> 3.27"
    }
  }
  required_version = ">= 0.14.9"
}

provider "aws" {
  profile = "default"
  region  = "us-west-1"
}

resource "aws_s3_bucket" "bucket" {
  bucket = "${local.database_s3_bucket_name}"
}

# Ensure no public access is possible
resource "aws_s3_bucket_public_access_block" "bucket" {
  bucket = aws_s3_bucket.bucket.id

  block_public_acls       = true
  block_public_policy     = true
  ignore_public_acls      = true
  restrict_public_buckets = true
}
"""

main_tf.write(main_body)
main_tf.close()
variables_tf.write(variables)
variables_tf.close()

In [5]:
%%bash
terraform init 


Initializing the backend...

Initializing provider plugins...
- Finding hashicorp/aws versions matching "~> 3.27"...
- Installing hashicorp/aws v3.71.0...
- Installed hashicorp/aws v3.71.0 (signed by HashiCorp)

Terraform has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


In [6]:
%%bash
cd terraform_tutorial
terraform plan


Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # aws_s3_bucket.bucket will be created
  + resource "aws_s3_bucket" "bucket" {
      + acceleration_status         = (known after apply)
      + acl                         = "private"
      + arn                         = (known after apply)
      + bucket                      = "yellow-taxi-ny-983794"
      + bucket_domain_name          = (known after apply)
      + bucket_regional_domain_name = (known after apply)
      + force_destroy               = false
      + hosted_zone_id              = (known after apply)
      + id                          = (known after apply)
      + region                      = (known after apply)
      + request_payer               = (known after apply)
      + tags_all                    = (known after apply)
      + website_domain              = (known

bash: line 1: cd: terraform_tutorial: No such file or directory


In [7]:
%%bash
terraform apply -auto-approve


Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # aws_s3_bucket.bucket will be created
  + resource "aws_s3_bucket" "bucket" {
      + acceleration_status         = (known after apply)
      + acl                         = "private"
      + arn                         = (known after apply)
      + bucket                      = "yellow-taxi-ny-983794"
      + bucket_domain_name          = (known after apply)
      + bucket_regional_domain_name = (known after apply)
      + force_destroy               = false
      + hosted_zone_id              = (known after apply)
      + id                          = (known after apply)
      + region                      = (known after apply)
      + request_payer               = (known after apply)
      + tags_all                    = (known after apply)
      + website_domain              = (known

╷
│ Error: Error creating S3 bucket: BucketAlreadyOwnedByYou: Your previous request to create the named bucket succeeded and you already own it.
│ 	status code: 409, request id: T99K5ZSEMCMSVZCV, host id: SFVFMqR0f35R6wrDGc/NfdZFW1x4EE7JDIO/mpe//kjP4q5noAqVP2Hz77iyHMon8nfJKYm5Em8=
│ 
│   with aws_s3_bucket.bucket,
│   on main.tf line 17, in resource "aws_s3_bucket" "bucket":
│   17: resource "aws_s3_bucket" "bucket" {
│ 
╵


In [8]:
%%bash
wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2021-01.csv -o /dev/null

In [9]:
%%bash
aws s3 cp /content/terraform_tutorial/yellow_tripdata_2021-01.csv s3://yellow-taxi-ny-983794
aws s3 ls s3://yellow-taxi-ny-983794/

upload: ./yellow_tripdata_2021-01.csv to s3://yellow-taxi-ny-983794/yellow_tripdata_2021-01.csv
2022-01-08 02:10:55          0 cb7da54e-d080-456f-bedd-4787eb3124a6.txt
2022-01-09 00:03:25  125981363 yellow_tripdata_2021-01.csv


https://www.xerris.com/insights/building-modern-data-warehouses-with-s3-glue-and-athena-part-3/

In [42]:
athena_tf_body = """

resource "aws_athena_database" "yellow_taxi_ny" {
  name   = "yellow_taxi_ny"
  bucket = aws_s3_bucket.bucket.bucket
}

resource "aws_s3_bucket" "query_results" {
  bucket = "${local.query_results_s3_bucket_name}"
  lifecycle_rule {
    enabled = true
    expiration {
      days = 1
    }
  }
}

# Ensure no public access is possible
resource "aws_s3_bucket_public_access_block" "query_results_bucket_block" {
  bucket = aws_s3_bucket.query_results.id

  block_public_acls       = true
  block_public_policy     = true
  ignore_public_acls      = true
  restrict_public_buckets = true
}

resource "aws_athena_workgroup" "workgroup" {
  name = "my_workgroup"
  configuration {
    enforce_workgroup_configuration    = true
    publish_cloudwatch_metrics_enabled = true

    result_configuration {
      output_location = "s3://${aws_s3_bucket.query_results.bucket}/output/"
    }
  }
}

resource "aws_glue_crawler" "crawler" {
  database_name = aws_athena_database.yellow_taxi_ny.name
  name          = "yellow_taxi_ny_crawler"
  role          = aws_iam_role.yellow_taxi_nydb_role.arn

  s3_target {
    path = "s3://${aws_s3_bucket.bucket.bucket}"
  }
}
"""

iam_tf_body = """

resource "aws_iam_role" "yellow_taxi_nydb_role" {
  name               = "yellow_taxi_nydb_role"
  assume_role_policy = data.aws_iam_policy_document.glue-assume-role-policy.json
}

data "aws_iam_policy_document" "glue-assume-role-policy" {
  statement {
    actions = ["sts:AssumeRole"]
    principals {
      type        = "Service"
      identifiers = ["glue.amazonaws.com"]
    }
  }
}

resource "aws_iam_policy" "policy" {
  name        = "yellow_taxi_nydb"
  description = "yellow_taxi_nydb"
  policy      = data.aws_iam_policy_document.policy-document.json

}

data "aws_iam_policy_document" "policy-document" {
  statement {
    actions = [
    "s3:GetBucketLocation", "s3:ListBucket", "s3:ListAllMyBuckets", 
    "s3:GetBucketAcl", "s3:GetObject"]
    resources = [
      "arn:aws:s3:::yellow-taxi-ny-983794",
      "arn:aws:s3:::yellow-taxi-ny-983794/*"
    ]
  }
}

resource "aws_iam_role_policy_attachment" "policy-attachment" {
  role       = aws_iam_role.yellow_taxi_nydb_role.name
  policy_arn = aws_iam_policy.policy.arn
}

resource "aws_iam_role_policy_attachment" "glue-service-role-attachment" {
  role       = aws_iam_role.yellow_taxi_nydb_role.name
  policy_arn = data.aws_iam_policy.AWSGlueServiceRole.arn
}

data "aws_iam_policy" "AWSGlueServiceRole" {
  arn = "arn:aws:iam::aws:policy/service-role/AWSGlueServiceRole"
}

"""

with open("athena.tf","w") as athena_tf:
  athena_tf.writelines(athena_tf_body)

with open("iam.tf","w") as iam_tf:
  iam_tf.writelines(iam_tf_body)

In [43]:
%%bash
#terraform plan
terraform apply -auto-approve

aws_s3_bucket.query_results: Refreshing state... [id=yellow-taxi-ny-983794-workgroup-query-results]
aws_s3_bucket.bucket: Refreshing state... [id=yellow-taxi-ny-983794]
aws_iam_role.yellow_taxi_nydb_role: Refreshing state... [id=yellow_taxi_nydb_role]
aws_iam_policy.policy: Refreshing state... [id=arn:aws:iam::736682772784:policy/yellow_taxi_nydb]
aws_iam_role_policy_attachment.policy-attachment: Refreshing state... [id=yellow_taxi_nydb_role-20220108021055016300000002]
aws_iam_role_policy_attachment.glue-service-role-attachment: Refreshing state... [id=yellow_taxi_nydb_role-20220108021055025600000003]
aws_s3_bucket_public_access_block.bucket: Refreshing state... [id=yellow-taxi-ny-983794]
aws_athena_database.yellow_taxi_ny: Refreshing state... [id=yellow_taxi_ny]
aws_athena_workgroup.workgroup: Refreshing state... [id=my_workgroup]
aws_s3_bucket_public_access_block.query_results_bucket_block: Refreshing state... [id=yellow-taxi-ny-983794-workgroup-query-results]
aws_glue_crawler.crawle

In [44]:
%%bash
aws glue start-crawler --name yellow_taxi_ny_crawler

In [25]:
# install a json string processor (jq)
#!sudo apt install jq >> /dev/null

In [45]:
%%bash
aws glue get-crawler --name yellow_taxi_ny_crawler

{
    "Crawler": {
        "Name": "yellow_taxi_ny_crawler",
        "Role": "yellow_taxi_nydb_role",
        "Targets": {
            "S3Targets": [
                {
                    "Path": "s3://yellow-taxi-ny-983794",
                    "Exclusions": []
                }
            ],
            "JdbcTargets": [],
            "MongoDBTargets": [],
            "DynamoDBTargets": [],
            "CatalogTargets": []
        },
        "DatabaseName": "yellow_taxi_ny",
        "Classifiers": [],
        "RecrawlPolicy": {
            "RecrawlBehavior": "CRAWL_EVERYTHING"
        },
        "SchemaChangePolicy": {
            "UpdateBehavior": "UPDATE_IN_DATABASE",
            "DeleteBehavior": "DEPRECATE_IN_DATABASE"
        },
        "LineageConfiguration": {
            "CrawlerLineageSettings": "DISABLE"
        },
        "State": "RUNNING",
        "CrawlElapsedTime": 32451,
        "CreationTime": "2022-01-08T02:11:01+00:00",
        "LastUpdated": "2022-01-08T02:11:01+0

In [46]:
%%bash
crawler_details=$(aws glue get-crawler --name yellow_taxi_ny_crawler)
#aws glue get-tables --database-name yellow_taxi_ny
#echo $crawler_details
status=$(jq .Crawler.State <<< $crawler_details)
echo $status

"STOPPING"


In [49]:
%%bash
#aws glue list-databases
aws glue get-tables --database-name 'yellow_taxi_ny'

{
    "TableList": [
        {
            "Name": "yellow_taxi_ny_983794",
            "DatabaseName": "yellow_taxi_ny",
            "Owner": "owner",
            "CreateTime": "2022-01-08T02:23:52+00:00",
            "UpdateTime": "2022-01-08T02:23:52+00:00",
            "LastAccessTime": "2022-01-08T02:23:52+00:00",
            "Retention": 0,
            "StorageDescriptor": {
                "Columns": [
                    {
                        "Name": "vendorid",
                        "Type": "bigint"
                    },
                    {
                        "Name": "tpep_pickup_datetime",
                        "Type": "string"
                    },
                    {
                        "Name": "tpep_dropoff_datetime",
                        "Type": "string"
                    },
                    {
                        "Name": "passenger_count",
                        "Type": "bigint"
                    },
                    {
             

In [61]:
%%bash
query_execution_id=$(jq .QueryExecutionId <<< $(aws athena start-query-execution --work-group my_workgroup --query-string "SELECT * FROM yellow_taxi_ny.yellow_taxi_ny_983794 limit 100;"))
echo $query_execution_id

"d2736a78-5fa5-4372-bf17-2b579dfdfaec"


In [71]:
%%bash
query_execution_json=$(aws athena get-query-execution --query-execution-id d2736a78-5fa5-4372-bf17-2b579dfdfaec)
jq .QueryExecution.Status <<< $query_execution_json

{
  "State": "SUCCEEDED",
  "SubmissionDateTime": "2022-01-08T02:45:25.521000+00:00",
  "CompletionDateTime": "2022-01-08T02:45:26.265000+00:00"
}


In [63]:
#!aws athena get-query-results --query-execution-id c561bb4d-d75c-4586-8e9b-e3cd9347bcf3
%%bash
aws s3 mv s3://yellow-taxi-ny-983794-workgroup-query-results/output/d2736a78-5fa5-4372-bf17-2b579dfdfaec.csv /content/results.csv
cat /content/results.csv

move: s3://yellow-taxi-ny-983794-workgroup-query-results/output/d2736a78-5fa5-4372-bf17-2b579dfdfaec.csv to ../results.csv
"vendorid","tpep_pickup_datetime","tpep_dropoff_datetime","passenger_count","trip_distance","ratecodeid","store_and_fwd_flag","pulocationid","dolocationid","payment_type","fare_amount","extra","mta_tax","tip_amount","tolls_amount","improvement_surcharge","total_amount","congestion_surcharge"
"2","2021-01-10 19:05:32","2021-01-10 19:14:31","2","2.05","1","N","161","246","1","9.0","0.0","0.5","3.69","0.0","0.3","15.99","2.5"
"2","2021-01-10 19:03:03","2021-01-10 19:08:49","1","1.03","1","N","237","262","1","6.0","0.0","0.5","1.0","0.0","0.3","10.3","2.5"
"2","2021-01-10 19:17:12","2021-01-10 19:29:53","1","2.78","1","N","236","166","1","11.5","0.0","0.5","2.96","0.0","0.3","17.76","2.5"
"2","2021-01-10 19:33:13","2021-01-10 19:45:07","1","1.78","1","N","166","75","2","10.0","0.0","0.5","0.0","0.0","0.3","10.8","0.0"
"2","2021-01-10 19:52:14","2021-01-10 19:56:17","1"

## Data Visualization

In [10]:
! pip install -q folium

In [11]:
import pandas as pd
import folium
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime

In [12]:
df_full_dataset = pd.read_csv('/content/terraform_tutorial/yellow_tripdata_2021-01.csv')

#df_train = pd.read_csv('../input/nyc-taxi-trip-duration/train.zip').drop(columns=['trip_duration', 'dropoff_datetime'])
#df_test = pd.read_csv('../input/nyc-taxi-trip-duration/test.zip')
#df = pd.concat([df_train, df_test], sort=False, ignore_index=True)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
df_full_dataset.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2021-01-01 00:30:10,2021-01-01 00:36:12,1.0,2.10,1.0,N,142,43,2.0,8.0,3.0,0.5,0.00,0.0,0.3,11.80,2.5
1,1.0,2021-01-01 00:51:20,2021-01-01 00:52:19,1.0,0.20,1.0,N,238,151,2.0,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0
2,1.0,2021-01-01 00:43:30,2021-01-01 01:11:06,1.0,14.70,1.0,N,132,165,1.0,42.0,0.5,0.5,8.65,0.0,0.3,51.95,0.0
3,1.0,2021-01-01 00:15:48,2021-01-01 00:31:01,0.0,10.60,1.0,N,138,132,1.0,29.0,0.5,0.5,6.05,0.0,0.3,36.35,0.0
4,2.0,2021-01-01 00:31:49,2021-01-01 00:48:21,1.0,4.94,1.0,N,68,33,1.0,16.5,0.5,0.5,4.06,0.0,0.3,24.36,2.5


In [15]:
df_full_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1369765 entries, 0 to 1369764
Data columns (total 18 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   VendorID               1271413 non-null  float64
 1   tpep_pickup_datetime   1369765 non-null  object 
 2   tpep_dropoff_datetime  1369765 non-null  object 
 3   passenger_count        1271413 non-null  float64
 4   trip_distance          1369765 non-null  float64
 5   RatecodeID             1271413 non-null  float64
 6   store_and_fwd_flag     1271413 non-null  object 
 7   PULocationID           1369765 non-null  int64  
 8   DOLocationID           1369765 non-null  int64  
 9   payment_type           1271413 non-null  float64
 10  fare_amount            1369765 non-null  float64
 11  extra                  1369765 non-null  float64
 12  mta_tax                1369765 non-null  float64
 13  tip_amount             1369765 non-null  float64
 14  tolls_amount      

In [18]:
df_full_dataset.tpep_pickup_datetime = pd.to_datetime(df_full_dataset.tpep_pickup_datetime, format='%Y-%m-%d %H:%M:%S')
df_full_dataset['month'] = df_full_dataset.tpep_pickup_datetime.apply(lambda x: x.month)
df_full_dataset['week'] = df_full_dataset.tpep_pickup_datetime.apply(lambda x: x.week)
df_full_dataset['day'] = df_full_dataset.tpep_pickup_datetime.apply(lambda x: x.day)
df_full_dataset['hour'] = df_full_dataset.tpep_pickup_datetime.apply(lambda x: x.hour)

In [20]:
df_full_dataset.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,month,week,day,hour
0,1.0,2021-01-01 00:30:10,2021-01-01 00:36:12,1.0,2.10,1.0,N,142,43,2.0,8.0,3.0,0.5,0.00,0.0,0.3,11.80,2.5,1,53,1,0
1,1.0,2021-01-01 00:51:20,2021-01-01 00:52:19,1.0,0.20,1.0,N,238,151,2.0,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0,1,53,1,0
2,1.0,2021-01-01 00:43:30,2021-01-01 01:11:06,1.0,14.70,1.0,N,132,165,1.0,42.0,0.5,0.5,8.65,0.0,0.3,51.95,0.0,1,53,1,0
3,1.0,2021-01-01 00:15:48,2021-01-01 00:31:01,0.0,10.60,1.0,N,138,132,1.0,29.0,0.5,0.5,6.05,0.0,0.3,36.35,0.0,1,53,1,0
4,2.0,2021-01-01 00:31:49,2021-01-01 00:48:21,1.0,4.94,1.0,N,68,33,1.0,16.5,0.5,0.5,4.06,0.0,0.3,24.36,2.5,1,53,1,0


In [21]:
def generateBaseMap(default_location=[40.693943, -73.985880], default_zoom_start=11):
    """
    location: Define the default location to zoom at when rendering the map
    zoom_start: The zoom level that the map will default to when rendering the map
    control_scale: Shows the map scale for a given zoom level
    """
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [22]:
base_map = generateBaseMap()
base_map

https://www.kaggle.com/faridrizqis/folium-for-map-visualization

## Analysis Question:
The city is planning to install taxi stops, locations throughout the city where people can get picked up/dropped off by cabs and wait for cabs to pick them up. This initiative is aiming to:

1. Reduce the amount of taxis that stops at the bicycle lanes 
2. Making it easier and safer for the citizens of the city to get a cab especially during night time 
3. Allowing taxi cab drivers to know where to go when they want to find riders
4. Allowing potential riders to have a point where they can go to when they are looking for taxis.

You are tasked with suggesting the best locations for these taxi stops, and also generating insights about the pattern of rides amount throughout the day across the city.

KeyError: ignored

http://ramondario.com/processing-nyc-taxi-data-part-1-downloading.html